Мышковец С.А., v.01 07.12.2022

Решение задачи: 

1. Из текстуальных данных с использованием алгоритмов кластеризации найти похожие. 
2. В выводах обосновать какие метрики и алгоритмы лучше использовать для решения вышеизложенных задач.

Вывод:

1. Построить рекомендательную систему при помощи NMF можнo.
2. Непонятно, как считать качество кластеризации.
3. Возможно, результаты с данные, состоящими из статей с заголовками, были бы более показательными.


In [1]:
import numpy as np 
import warnings
warnings.simplefilter('ignore')

np.random.seed(0)

**Загрузить данные**

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import pandas as pd

In [3]:
dt_train = fetch_20newsgroups_vectorized()

In [4]:
dt_train.data

<11314x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [5]:
dt_train.data = dt_train.data[0:2000]

In [6]:
dt_train.target = dt_train.target[0:2000]

In [7]:
dt_train.target

array([17,  7, 10, ...,  0,  5, 15])

In [8]:
dt_train.data.shape

(2000, 130107)

In [9]:
dt_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## NMF

In [10]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=20)
nmf_features = nmf.fit_transform(dt_train.data)

In [11]:
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)

In [12]:
df = pd.DataFrame(norm_features)

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.523256,0.208707,0.054219,0.533341,0.226444,0.067211,0.000000,0.431952,0.018511,0.054236,0.267914,0.045444,0.007448,0.069502,0.016820,0.097948,0.048394,0.109159,0.216165,0.000000
1,0.457182,0.000000,0.317233,0.477907,0.400789,0.137448,0.133519,0.109833,0.053530,0.000000,0.191928,0.023590,0.000000,0.150374,0.074645,0.224444,0.124763,0.340195,0.049216,0.000000
2,0.372057,0.446175,0.028971,0.266913,0.341334,0.000000,0.009200,0.256710,0.139460,0.000000,0.100776,0.187630,0.127544,0.346382,0.031950,0.122750,0.000000,0.436784,0.000000,0.000000
3,0.401612,0.437214,0.222003,0.295911,0.374592,0.034284,0.002134,0.168786,0.147871,0.176343,0.187656,0.245126,0.037057,0.350835,0.034098,0.000000,0.103076,0.150957,0.158307,0.091404
4,0.065374,0.025035,0.146306,0.307479,0.227151,0.128689,0.000000,0.074658,0.642513,0.000000,0.248159,0.251663,0.147070,0.061447,0.320112,0.252210,0.239826,0.000000,0.136658,0.000000


In [14]:
# Найдем новости, похожии на новость с индексом 60

current_news = norm_features[60, :]
similarities = norm_features.dot(current_news)

In [15]:
current_news = df.loc[60]

In [16]:
similarities = df.dot(current_news)

In [17]:
similarities.nlargest(10)

60      1.000000
1827    0.945566
580     0.923661
1135    0.913098
159     0.912252
453     0.898702
675     0.897974
1525    0.895453
684     0.886819
1511    0.885319
dtype: float64

In [18]:
sim_list = similarities.nlargest(6).index
sim_list_indices = [i for i in sim_list]

In [19]:
sim_list_indices

[60, 1827, 580, 1135, 159, 453]

In [20]:
res = []
for i in sim_list_indices:
    res.append(dt_train.target[i])
    print(dt_train.target[i])
print(res)
print((res.count(res[0])-1)/5*100)

11
11
11
11
7
3
[11, 11, 11, 11, 7, 3]
60.0


In [21]:
# Обработаем весь новостной массив и посмотрим % точного определения темы из 5 наиболее вероятных вариантов.

result = []
for i in range(dt_train.data[0:2000].shape[0]):
    current_news = norm_features[i, :]
    similarities = norm_features.dot(current_news)
    current_news = df.loc[i]
    similarities = df.dot(current_news)
    sim_list = similarities.nlargest(6).index
    sim_list_indices = [i for i in sim_list]
    res = []
    for i in sim_list_indices:
        res.append(dt_train.target[i])
    result.append((res.count(res[0])-1)/5*100)

In [22]:
np.mean(result)

12.37

## Не знаю, как тут посчитать метрики, но больше похоже на случайное угадывание. Среднее точное попадание в тему 12.4 %. Но, учитывая, что есть темы близкие по содержанию (e.g. 'alt.atheism', 'soc.religion.christian', 'talk.religion.misc'), возможно, что результат не так и плох.